# Домашнее задание 1. Разрешение семантической многозначности (WSD).

In [ ]:
!pip install git+https://github.com/lopuhin/python-adagram.git

In [ ]:
!pip install corus

In [ ]:
!pip install pymorphy3

In [ ]:
!pip install simple_elmo

In [5]:
# библиотеки для работы с эмбеддингами
import adagram
from simple_elmo import ElmoModel

# обработка данных
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
from nltk import tokenize
tokenizer = tokenize.WordPunctTokenizer()

morph = MorphAnalyzer()

stops = stopwords.words('russian')

## 1. Выберем 5 неоднозначных лексем и соберём для них словарные значения.

Для получения словарных значений воспользуемся МАС. Значения собирались вручную.

In [8]:
words = ["марка", "лук", "заработать", "звезда", "жалоба"]

In [9]:
dictionary = {"марка" : ["Знак оплаты почтовых, гербовых и некоторых других сборов в виде маленькой бумажки с обозначением цены и, обычно, с каким-л. изображением.",
                         "Значок, жетон для последующей оплаты.",
                         "Знак, клеймо на товарах, изделиях с обозначением места изготовления, качества и др.",
                         "Сорт, тип изделия, качество товара.",
                         "Денежная единица ГДР и ФРГ, равная 100 пфеннигам",
                         "Денежная единица Финляндии, равная 100 пенни.",
                         "Крупный пограничный административный округ во главе с маркграфом во Франкском государстве в 8—9 вв."],
              "лук" : ["Огородное растение сем. лилейных, овощ.",
                       "Ручное оружие для метания стрел в виде дуги, стянутой тетивой."],
              "заработать" : ["Приобрести работой, получить за работу.",
                              "Начать работать."],
              "звезда" : ["Небесное тело, состоящее из раскаленных газов (плазмы), по своей природе сходное с Солнцем и представляющееся взору человека на ночном небе светящейся точкой.",
                          "О человеке, прославившемся в какой-л. сфере деятельности; о знаменитости.",
                          "Геометрическая фигура с остроконечными выступами, равномерно расположенными по окружности; фигура с лучами, исходящими от центра."],
              "жалоба" : ["Официальное заявление о незаконном или неправильном действии какого-л. лица, учреждения, организации.",
                          "Выражение неудовольствия, печали, сетование по поводу неприятностей, боли и т. п."]}


In [10]:
def preprocess_text(text):
    '''
    Предобработка текста:
    * лемматизация
    * очистка от пунктуации и стоп-слов
    '''
    lemmas = [morph.parse(word.lower())[0].normal_form for word in tokenizer.tokenize(text) if word.isalpha() and
              word not in stops]
    return lemmas

In [11]:
df = []

for word, definitions in dictionary.items():
    for n, definition in enumerate(definitions, start=1):
        df.append((word, n, definition, preprocess_text(definition)))

definitions_df = pd.DataFrame(df, columns=['word', 'id_def', 'definition', 'lemmatized_sentence'])

In [12]:
definitions_df.head(2)

,word,id_def,definition,lemmatized_sentence
0,марка,1,"Знак оплаты почтовых, гербовых и некоторых дру...","[знак, оплата, почтовый, гербовый, некоторый, ..."
1,марка,2,"Значок, жетон для последующей оплаты.","[значок, жетон, последующий, оплата]"


## 2. Выберем несколько корпусов коллекции Corus и извлечём оттуда все предложения, где содержатся наши неоднозначные слова.

In [ ]:
! wget https://github.com/UniversalDependencies/UD_Russian-GSD/raw/master/ru_gsd-ud-dev.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-GSD/raw/master/ru_gsd-ud-test.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-PUD/raw/master/ru_pud-ud-test.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-dev.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-test.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-Taiga/raw/master/ru_taiga-ud-dev.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-Taiga/raw/master/ru_taiga-ud-test.conllu
! wget https://github.com/UniversalDependencies/UD_Russian-Taiga/raw/master/ru_taiga-ud-train.conllu

In [15]:
from corus import load_ud_gsd, load_ud_pud, load_ud_taiga, load_ud_syntag

In [16]:
contexts = {word: [] for word in words }

In [17]:
def get_contexts(corpus):
    for sentence in corpus:
        lemmas = [token.lemma for token in sentence.tokens]
        lemmas = [x for x in lemmas if x != None]
    for word in words:
        if word in lemmas:
            contexts[word].append((sentence.text, lemmas))

In [18]:
corpora = [load_ud_gsd('ru_gsd-ud-dev.conllu'),
           load_ud_gsd('ru_gsd-ud-test.conllu'),
           load_ud_pud('ru_pud-ud-test.conllu'),
           load_ud_syntag('ru_syntagrus-ud-dev.conllu'),
           load_ud_syntag('ru_syntagrus-ud-test.conllu'),
           load_ud_taiga('ru_taiga-ud-dev.conllu'),
           load_ud_taiga('ru_taiga-ud-test.conllu')]

for i in corpora:
    get_contexts(i)

In [19]:
for word in words:
    print(word, len(contexts[word]))

марка 10
лук 5
заработать 16
звезда 38
жалоба 4


In [20]:
contexts = [(word, sentence, lemmatized_sentence)
            for word, sentences in contexts.items()
            for sentence, lemmatized_sentence in sentences]

df = pd.DataFrame(contexts, columns=['word', 'sentence', 'lemmatized_sentence'])

In [21]:
df.tail(3)

,word,sentence,lemmatized_sentence
70,жалоба,"Вновь на Макарова идут жалобы, что он не спосо...","[вновь, на, Макаров, идти, жалоба, ,, что, он,..."
71,жалоба,"- Скажите мне, кому я могу подать официальную ...","[-, сказать, я, ,, кто, я, мочь, подать, офици..."
72,жалоба,14.06.2016 я получил уведомление из суда содер...,"[14.06.2016, я, получить, уведомление, из, суд..."


## 3. Разобьём полученные контексты по значениям двумя способами (AdaGram и ELMo)

In [ ]:
!wget 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib'

In [23]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [ ]:
ada_list_context = []

for index, row in df.iterrows():
    ada_list_context.append(vm.disambiguate(row.word, row.lemmatized_sentence).argmax())

df = df.assign(ada=ada_list_context)

In [25]:
df

,word,sentence,lemmatized_sentence,ada
0,марка,С октября 1982 по октябрь 1987 года выпускалас...,"[с, октябрь, 1982, по, октябрь, 1987, год, вып...",0
1,марка,"Расследование началось в июне, и главным образ...","[расследование, начаться, в, июнь, ,, и, главн...",1
2,марка,Ученые Института неорганической химии совместн...,"[ученый, Институт, неорганический, химия, совм...",3
3,марка,"Мировые марки, коренные изменения в структуре ...","[мировой, марка, ,, коренной, изменение, в, ст...",3
4,марка,С наступлением темноты сверкающие поверхности ...,"[с, наступление, темнота, сверкать, поверхност...",4
...,...,...,...,...
68,звезда,Какой мудак из Челябинска пообещал своей бабе ...,"[какой, мудак, из, Челябинск, пообещать, свой,...",3
69,жалоба,Но растущий средний класс Китая необычайно акт...,"[но, расти, средний, класс, Китай, необычайно,...",1
70,жалоба,"Вновь на Макарова идут жалобы, что он не спосо...","[вновь, на, Макаров, идти, жалоба, ,, что, он,...",2
71,жалоба,"- Скажите мне, кому я могу подать официальную ...","[-, сказать, я, ,, кто, я, мочь, подать, офици...",2


3.2 Elmo

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/196.zip

In [ ]:
model = ElmoModel()
model.load('196.zip')

In [28]:
df['lemmatized_sentence'] = df['lemmatized_sentence'].apply(lambda x: ' '.join(x))

In [29]:
def get_elmo_vectors(word, contexts, model):
    tokens = [c.split() for c in contexts]
    all_vectors = model.get_elmo_vectors(tokens)
    word_vecs = []
    for i in range(len(contexts)):
        try:
            word_vecs.append(all_vectors[i][tokens[i].index(word)])
        except ValueError:
            continue
    return word_vecs

In [30]:
result_list = []

for word in words:
    elmo_vectors = get_elmo_vectors(word, list(df[df['word'] == word]['lemmatized_sentence']), model)
    result_list.extend(elmo_vectors)

In [31]:
df['elmo_vectors'] = result_list

In [32]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

k_models = dict()

In [ ]:
for word in df['word'].unique():
    elmo_vectors = df[df['word'] == word]['elmo_vectors'].tolist()
    if len(elmo_vectors) > 0:
        X = [v.tolist() for v in elmo_vectors]
        k_range = range(2, 4)
        silhouette_scores = []
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42)
            clusters = kmeans.fit_predict(X)
            silhouette_scores.append(silhouette_score(X, clusters))
        optimal_k = k_range[silhouette_scores.index(max(silhouette_scores))]
        kmeans = KMeans(n_clusters=optimal_k, random_state=42)
        k_models[word] = kmeans
        clusters = kmeans.fit_predict(X)
        clusters = clusters.astype(int) + 1
        df.loc[df['word'] == word, 'elmo_cluster'] = clusters

In [72]:
df = df.drop(columns=['elmo_vectors'])

In [73]:
df.head()

,word,sentence,lemmatized_sentence,ada,elmo_cluster
0,марка,С октября 1982 по октябрь 1987 года выпускалась вторая серия авиапочтовых марок с изображением стилизованного самолёта над письмом.,с октябрь 1982 по октябрь 1987 год выпускаться второй серия авиапочтовый марка с изображение стилизованный самолет над письмо .,0,2.0
1,марка,"Расследование началось в июне, и главным образом внимание было сосредоточено на господине Винтеркорне и председателе совета марки VW Херберте Диессе, который все еще работает в компании.","расследование начаться в июнь , и главный образ внимание быть сосредоточить на господин Винтеркорн и председатель совет марка VW Херберт Диесс , который все еще работать в компания .",1,2.0
2,марка,"Ученые Института неорганической химии совместно со своими коллегами из ЧССР и ГДР разработали рецептуру новых марок НЛС, обладающих антикоррозионной стойкостью.","ученый Институт неорганический химия совместно с свой коллега из ЧССР и ГДР разработать рецептура новый марка НЛС , обладать антикоррозионный стойкость .",3,2.0
3,марка,"Мировые марки, коренные изменения в структуре производства и распределения товаров быстро насыщают уникальное местное пространство элементами, не принадлежащими какому-либо определенному культурному контексту.","мировой марка , коренной изменение в структура производство и распределение товар быстро насыщать уникальный местный пространство элемент , не принадлежать какой-либо определенный культурный контекст .",3,2.0
4,марка,"С наступлением темноты сверкающие поверхности наполняются движущимися цветными огнями, среди которых в глаза бросаются названия мировых марок одежды, косметики и ювелирных изделий.","с наступление темнота сверкать поверхность наполняться двигаться цветной огонь , среди который в глаз бросаться название мировой марка одежда , косметика и ювелирный изделие .",4,2.0


## 4. Возьмем словарные толкования для каждого значения и автоматически припишем им значение

3.1 AdaGram

In [ ]:
ada_list = []

for index, row in definitions_df.iterrows():
    ada_list.append(vm.disambiguate(row.word, row.lemmatized_sentence).argmax())

In [36]:
definitions_df = definitions_df.assign(ada=ada_list)

In [37]:
definitions_df

,word,id_def,definition,lemmatized_sentence,ada
0,марка,1,"Знак оплаты почтовых, гербовых и некоторых дру...","[знак, оплата, почтовый, гербовый, некоторый, ...",0
1,марка,2,"Значок, жетон для последующей оплаты.","[значок, жетон, последующий, оплата]",0
2,марка,3,"Знак, клеймо на товарах, изделиях с обозначени...","[знак, клеймо, товар, изделие, обозначение, ме...",3
3,марка,4,"Сорт, тип изделия, качество товара.","[сорт, тип, изделие, качество, товар]",3
4,марка,5,"Денежная единица ГДР и ФРГ, равная 100 пфеннигам","[денежный, единица, гдр, фрг, равный, пфенниг]",0
5,марка,6,"Денежная единица Финляндии, равная 100 пенни.","[денежный, единица, финляндия, равный, пенни]",0
6,марка,7,Крупный пограничный административный округ во ...,"[крупный, пограничный, административный, округ...",1
7,лук,1,"Огородное растение сем. лилейных, овощ.","[огородный, растение, сем, лилейный, овощ]",0
8,лук,2,"Ручное оружие для метания стрел в виде дуги, с...","[ручной, оружие, метание, стрела, вид, дуга, с...",1
9,заработать,1,"Приобрести работой, получить за работу.","[приобрести, работа, получить, работа]",1


Посмотрим, насколько автоматически приписаное значение соотвествует словарному

In [38]:
for word in words:
    print(vm.word_sense_probs(word))
    for i in vm.word_sense_probs(word):
        print(word, i[0], vm.sense_neighbors(word, i[0])[:5])

[(0, 0.20457747104062018), (1, 0.08050021918938104), (2, 0.14638336722088552), (3, 0.21531034614248035), (4, 0.3532277809950783)]


/usr/local/lib/python3.10/dist-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


марка 0 [('почтовый', 1, 0.828428), ('надпечатка', 0, 0.75926304), ('доплатной', 0, 0.7463568), ('штемпель', 0, 0.71790075), ('надпечаток', 0, 0.70800066)]
марка 1 [('марк', 0, 0.5678235), ('антоний', 1, 0.5616417), ('корнелия', 0, 0.56142837), ('публий', 0, 0.54651153), ('луция', 0, 0.54124033)]
марка 2 [('известный', 1, 0.6545491), ('бренд', 1, 0.61281645), ('торговый', 2, 0.6097513), ('цена', 4, 0.5665743), ('оригинал', 1, 0.56367534)]
марка 3 [('импортный', 1, 0.6165878), ('типоразмер', 0, 0.61335075), ('изготовитель', 1, 0.57336664), ('автомобиль', 2, 0.5723816), ('модель', 2, 0.5723381)]
марка 4 [('бренд', 2, 0.5125326), ('этикетка', 1, 0.50878817), ('импортный', 2, 0.5066383), ('дорогущий', 0, 0.495489), ('изделие', 3, 0.49242723)]
[(0, 0.27467387333364635), (1, 0.17510064513645113), (2, 0.25182447428150506), (3, 0.0852764349918363), (4, 0.2131232975158767)]
лук 0 [('морковь', 0, 0.8814602), ('помидор', 0, 0.879848), ('лук-порей', 0, 0.8786377), ('зелень', 0, 0.872901), ('нареза

In [39]:
definitions_df['my_ada'] = [0, 'NA', 2, 4, 'NA', 'NA', 'NA',
                            0, 1,
                            1, 0,
                            3, 2, 1,
                            0, 2]

In [40]:
definitions_df

,word,id_def,definition,lemmatized_sentence,ada,my_ada
0,марка,1,"Знак оплаты почтовых, гербовых и некоторых дру...","[знак, оплата, почтовый, гербовый, некоторый, ...",0,0
1,марка,2,"Значок, жетон для последующей оплаты.","[значок, жетон, последующий, оплата]",0,NA
2,марка,3,"Знак, клеймо на товарах, изделиях с обозначени...","[знак, клеймо, товар, изделие, обозначение, ме...",3,2
3,марка,4,"Сорт, тип изделия, качество товара.","[сорт, тип, изделие, качество, товар]",3,4
4,марка,5,"Денежная единица ГДР и ФРГ, равная 100 пфеннигам","[денежный, единица, гдр, фрг, равный, пфенниг]",0,NA
5,марка,6,"Денежная единица Финляндии, равная 100 пенни.","[денежный, единица, финляндия, равный, пенни]",0,NA
6,марка,7,Крупный пограничный административный округ во ...,"[крупный, пограничный, административный, округ...",1,NA
7,лук,1,"Огородное растение сем. лилейных, овощ.","[огородный, растение, сем, лилейный, овощ]",0,0
8,лук,2,"Ручное оружие для метания стрел в виде дуги, с...","[ручной, оружие, метание, стрела, вид, дуга, с...",1,1
9,заработать,1,"Приобрести работой, получить за работу.","[приобрести, работа, получить, работа]",1,1


3.2 Elmo

Добавим определяемое слово в определение

In [41]:
result_def_list = []

for word in words:
    new_def = []
    for definition in list(definitions_df[definitions_df['word'] == word]['lemmatized_sentence']):
        def_full = word + ' - ' + ' '.join(definition)
        new_def.append(def_full)
    result_def_list.append(new_def)

In [42]:
result_def_list

[['марка - знак оплата почтовый гербовый некоторый другой сбор вид маленький бумажка обозначение цена обычно какой л изображение',
  'марка - значок жетон последующий оплата',
  'марка - знак клеймо товар изделие обозначение место изготовление качество др',
  'марка - сорт тип изделие качество товар',
  'марка - денежный единица гдр фрг равный пфенниг',
  'марка - денежный единица финляндия равный пенни',
  'марка - крупный пограничный административный округ глава маркграф франкский государство век'],
 ['лук - огородный растение сем лилейный овощ',
  'лук - ручной оружие метание стрела вид дуга стянуть тетива'],
 ['заработать - приобрести работа получить работа',
  'заработать - начать работать'],
 ['звезда - небесный тело состоять раскалить газ плазма свой природа сходный солнце представляться взор человек ночной небо светящийся точка',
  'звезда - о человек прославиться л сфера деятельность знаменитость',
  'звезда - геометрический фигура остроконечный выступ равномерно расположить о

In [43]:
elno_def = []

for i, word in enumerate(words):
    elmo_def_vectors = get_elmo_vectors(word, result_def_list[i], model)
    elno_def.append(kmeans.predict(elmo_def_vectors))

In [49]:
definitions_df['elmo'] =  [x for xs in elno_def for x in xs]

## 5. Выберем по 2-3 значения для слова и по 5 контекстов, которые автоматически были отнесены к этим значениям.

In [50]:
definitions_df

,word,id_def,definition,lemmatized_sentence,ada,my_ada,elmo
0,марка,1,"Знак оплаты почтовых, гербовых и некоторых дру...","[знак, оплата, почтовый, гербовый, некоторый, ...",0,0,0
1,марка,2,"Значок, жетон для последующей оплаты.","[значок, жетон, последующий, оплата]",0,NA,0
2,марка,3,"Знак, клеймо на товарах, изделиях с обозначени...","[знак, клеймо, товар, изделие, обозначение, ме...",3,2,0
3,марка,4,"Сорт, тип изделия, качество товара.","[сорт, тип, изделие, качество, товар]",3,4,0
4,марка,5,"Денежная единица ГДР и ФРГ, равная 100 пфеннигам","[денежный, единица, гдр, фрг, равный, пфенниг]",0,NA,0
5,марка,6,"Денежная единица Финляндии, равная 100 пенни.","[денежный, единица, финляндия, равный, пенни]",0,NA,0
6,марка,7,Крупный пограничный административный округ во ...,"[крупный, пограничный, административный, округ...",1,NA,0
7,лук,1,"Огородное растение сем. лилейных, овощ.","[огородный, растение, сем, лилейный, овощ]",0,0,0
8,лук,2,"Ручное оружие для метания стрел в виде дуги, с...","[ручной, оружие, метание, стрела, вид, дуга, с...",1,1,0
9,заработать,1,"Приобрести работой, получить за работу.","[приобрести, работа, получить, работа]",1,1,0


Соотношение определённого автоматически значения и словарного:

In [74]:
pd.set_option('display.max_colwidth', None)

#### AdaGram

In [85]:
df[(df.word == 'звезда') & (df.ada == 0)].head(5)

,word,sentence,lemmatized_sentence,ada,elmo_cluster
32,звезда,"Пятиконечная звезда вписывается в окружность диаметром в 1/8 ширины флага, касающуюся верхней стороны квадрата.","пятиконечный звезда вписываться в окружность диаметр в 1/8 ширина флаг , касаться верхний сторона квадрат .",0,3.0
33,звезда,"Ещё в Падуе сделал большое число наблюдений астероидов, комет и покрытий звёзд.","еще в Падуя сделать большой число наблюдение астероид , комета и покрытие звезда .",0,1.0
34,звезда,"Она находится на расстоянии 16,29 астрономических единиц от звезды, и в толщине достигает 6,188 а. е.","она находиться на расстояние 16,29 астрономический единица от звезда , и в толщина достигать 6,188 а. е .",0,1.0
38,звезда,"Поводом к новому этапу в научных исследованиях Галилея послужило появление в 1604 году новой звезды, называемой сейчас сверхновой Кеплера.","повод к новый этап в научный исследование Галилей служить появление в 1604 год новый звезда , называть сейчас сверхновая Кеплер .",0,1.0
39,звезда,"Галилей открыл горы на Луне, Млечный путь распался на отдельные звёзды, но особенно поразили современников обнаруженные им 4 спутника Юпитера (1610).","Галилей открыть гора на Луна , млечный путь распасться на отдельный звезда , но особенно поразить современник обнаружить он 4 спутник Юпитер ( 1610 ) .",0,1.0


3 1 1 1 1

In [86]:
df[(df.word == 'звезда') & (df.ada == 1)].head(5)

,word,sentence,lemmatized_sentence,ada,elmo_cluster
31,звезда,"Был также награждён тремя орденами Красного Знамени, орденами Отечественной войны 1-й степени, Красной Звезды, ``За службу Родине в ВС СССР&#39;&#39; 3-й степени, рядом медалей.","быть также наградить три орден красный знамя , орден отечественный война 1-й степень , красный звезда , `` за служба родина в ВС СССР &#39;&#39; 3-й степень , ряд медаль .",1,2.0


3

In [87]:
df[(df.word == 'звезда') & (df.ada == 2)].head(5)

,word,sentence,lemmatized_sentence,ada,elmo_cluster
35,звезда,"На гонках появлялись звезды кино и телевидения, поддержать маленьких спортсменов (11-15 лет) собирались до 70000 человек.","на гонка появляться звезда кино и телевидение , поддержать маленький спортсмен ( 11 - 15 год ) собираться до 70000 человек .",2,1.0
36,звезда,Зрители помоложе воспринимают персонажи фильма как звезд рока.,зритель молодой воспринимать персонаж фильм как звезда рок .,2,1.0
37,звезда,"Эти молодые люди из Силиконовой долины в своем роде звезды, при том что они, как правило, не очень хороши собой.","этот молодой человек из силиконовый долина в свой род звезда , при то что они , как правило , не очень хороший себя .",2,1.0
41,звезда,"Французский король Генрих IV 20 апреля 1610 года, незадолго до своей гибели, просил Галилея открыть и для него какую-нибудь звезду.","французский король Генрих iv 20 апрель 1610 год , незадолго до свой гибель , просить Галилей открыть и для он какой-нибудь звезда .",2,1.0
52,звезда,"На передовиков и руководителей производственного объединения ""Белоруськалий"" щедро сыпались ордена и звезды.","на передовик и руководитель производственный объединение "" белоруськалий "" щедро сыпаться орден и звезда .",2,1.0


2 2 2 1 3

In [88]:
df[(df.word == 'звезда') & (df.ada == 3)].head(5)

,word,sentence,lemmatized_sentence,ada,elmo_cluster
45,звезда,"Млечный путь, который невооружённым глазом выглядит как сплошное сияние, распался на отдельные звёзды (что подтвердило догадку Демокрита), и стало видно громадное количество неизвестных ранее звёзд.","млечный путь , который невооруженный глаз выглядеть как сплошной сияние , распасться на отдельный звезда ( что подтвердить догадка Демокрит ) , и стать видный громадный количество неизвестный ранее звезда .",3,1.0
47,звезда,"Если бы не было терний, то к звездам мы пришли бы быстрее.","если бы не быть терние , то к звезда мы прийти бы быстро .",3,3.0
50,звезда,"Осенью звезды светились ярче, она выходила на свидания с ними, кутая плечи в материн рваный платок.","осень звезда светиться ярко , она выходить на свидание с они , кутать плечо в материн рваный платок .",3,3.0
51,звезда,"Ветер развевал серпантинные змеи, звезды плыли по реке.","ветер развевать серпантинный змея , звезда плыть по река .",3,3.0
54,звезда,"Высказывание греческого мудреца о том, что ""человек тем отличается от свиньи, что иногда поднимает голову и смотрит на звезды"", пришлось как раз кстати, и большинство признало, что ""заигрывать с Луной надо"".","высказывание греческий мудрец о то , что "" человек то отличаться от свинья , что иногда поднимать голова и смотреть на звезда "" , прийтись как раз кстати , и большинство признать , что "" заигрывать с Луна надо "" .",3,1.0


1 1 1 1 1

Вывод: одно и то же значение при автоматическом проставлении с помощью AdaGram оказалось разделено на 2 группы (0 и 3)

#### Elmo

In [84]:
df[(df.word == 'звезда') & (df.elmo_cluster == 0)].head(5)

,word,sentence,lemmatized_sentence,ada,elmo_cluster


In [79]:
df[(df.word == 'звезда') & (df.elmo_cluster == 1)].head(5)

,word,sentence,lemmatized_sentence,ada,elmo_cluster
33,звезда,"Ещё в Падуе сделал большое число наблюдений астероидов, комет и покрытий звёзд.","еще в Падуя сделать большой число наблюдение астероид , комета и покрытие звезда .",0,1.0
34,звезда,"Она находится на расстоянии 16,29 астрономических единиц от звезды, и в толщине достигает 6,188 а. е.","она находиться на расстояние 16,29 астрономический единица от звезда , и в толщина достигать 6,188 а. е .",0,1.0
35,звезда,"На гонках появлялись звезды кино и телевидения, поддержать маленьких спортсменов (11-15 лет) собирались до 70000 человек.","на гонка появляться звезда кино и телевидение , поддержать маленький спортсмен ( 11 - 15 год ) собираться до 70000 человек .",2,1.0
36,звезда,Зрители помоложе воспринимают персонажи фильма как звезд рока.,зритель молодой воспринимать персонаж фильм как звезда рок .,2,1.0
37,звезда,"Эти молодые люди из Силиконовой долины в своем роде звезды, при том что они, как правило, не очень хороши собой.","этот молодой человек из силиконовый долина в свой род звезда , при то что они , как правило , не очень хороший себя .",2,1.0


1 1 2 2 2

In [83]:
df[(df.word == 'звезда') & (df.elmo_cluster == 2)].head(5)

,word,sentence,lemmatized_sentence,ada,elmo_cluster
31,звезда,"Был также награждён тремя орденами Красного Знамени, орденами Отечественной войны 1-й степени, Красной Звезды, ``За службу Родине в ВС СССР&#39;&#39; 3-й степени, рядом медалей.","быть также наградить три орден красный знамя , орден отечественный война 1-й степень , красный звезда , `` за служба родина в ВС СССР &#39;&#39; 3-й степень , ряд медаль .",1,2.0


3

In [78]:
df[(df.word == 'звезда') & (df.elmo_cluster == 3)].head(5)

,word,sentence,lemmatized_sentence,ada,elmo_cluster
32,звезда,"Пятиконечная звезда вписывается в окружность диаметром в 1/8 ширины флага, касающуюся верхней стороны квадрата.","пятиконечный звезда вписываться в окружность диаметр в 1/8 ширина флаг , касаться верхний сторона квадрат .",0,3.0
47,звезда,"Если бы не было терний, то к звездам мы пришли бы быстрее.","если бы не быть терние , то к звезда мы прийти бы быстро .",3,3.0
50,звезда,"Осенью звезды светились ярче, она выходила на свидания с ними, кутая плечи в материн рваный платок.","осень звезда светиться ярко , она выходить на свидание с они , кутать плечо в материн рваный платок .",3,3.0
51,звезда,"Ветер развевал серпантинные змеи, звезды плыли по реке.","ветер развевать серпантинный змея , звезда плыть по река .",3,3.0
57,звезда,"На четвертый день, как правило, все уже поют песни в ресторанчике Толстого Джорджа, на пятый - смотрят на звезды затаив дыхание, на шестой - покупают местную симку и отключают московский телефон…","на четвертый день , как правило , все уже петь песня в ресторанчик толстый Джордж , на пятый - смотреть на звезда затаить дыхание , на шестой - покупать местный симка и отключать московский телефон …",2,3.0


3 1 1 1 1

Вывод: значения при использовании Elmo не раздробились, как было с AdaGram

## 6. Оценим accuracy

In [105]:
ada_accuracy = (4 + 1 + 3 + 5) / 16
elmo_accuracy =  (3 + 1 + 4) / 11

print(f'Adagram аccuracy: {ada_accuracy}')
print(f'Elmo аccuracy: {elmo_accuracy}')

Adagram аccuracy: 0.8125
Elmo аccuracy: 0.7272727272727273


## 7. Сравним результаты двух методов WSD

При приписывании меток словарным значениям AdaGram справился намного лучше, при приписывании значений контекстам результаты примерно одинаковые. AdaGram склонен выделять больше значений, иногда даже расщепляя одно и то же значение на два. Elmo выделяет меньше значений, что, вероятно, объясняется тем, что для Elmo текущая выборка содержит недостаточно примеров. Обе модели достаточно часто допускают ошибки. Возможно невысокие результаты моделей обусловлены небольшим количеством контекстов, и при увеличении выборки и длины контекста результаты станут более точными.